In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
!{sys.executable} -m pip install gym[all]

## Test out Generation

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)
# _ = model.to('cuda:0')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model.config.n_embd

768

In [17]:
text = ["tell me a joke"]
device = 'cpu'
# device = 'cuda:0'
encoded_input = tokenizer(text, return_tensors='pt', padding='max_length', max_length=10).to(device)

In [31]:
tokenizer.pad_token_id

50257

In [29]:
text = encoded_input['input_ids']
mask = encoded_input['attention_mask']
prompt = ["this is a prompt"]
prompt = tokenizer(prompt, return_tensors='pt').to(device)['input_ids']
text_zeroed_padding = (text + 1) * mask
text_trimmed = text[text_zeroed_padding.nonzero(as_tuple=True)]
torch.cat([text_trimmed, prompt[0]])
num_cols_all_padding = (text_zeroed_padding.sum(dim=0) == 0).sum().item()
print(text_zeroed_padding)
num_cols_all_padding

tensor([[33332,   503,   258,  9708,     0,     0,     0,     0,     0,     0]])


6

In [47]:
feedback = ['Be funnier']
encoded_input = tokenizer(text, return_tensors='pt')
encoded_feedback = tokenizer(text, return_tensors='pt')
text_length = encoded_input['input_ids'].shape[-1]
both = torch.cat([encoded_input['input_ids'], encoded_feedback['input_ids']], dim=1)
labels = both.clone()
labels[:, torch.arange(text_length)] = -100
labels

tensor([[ -100,  -100,  -100,  -100, 17250,   644,   338,   510]])

In [57]:
both

tensor([[17250,   644,   338,   510, 17250,   644,   338,   510]])

In [59]:
output = model(both, labels=labels)
output['loss']

tensor(4.0717, grad_fn=<NllLossBackward0>)

In [29]:
seq_len = encoded_input['input_ids'].shape[1]

In [66]:
outputs = model.generate(**encoded_input, max_length=seq_len+1, do_sample=True)
print(tokenizer.decode(outputs[0]))

Hi (


In [67]:
outputs

tensor([[17250,   357]], device='cuda:0')

In [71]:
for _ in range(5):
    outputs = model.generate(outputs, max_length=1, do_sample=True)
    print(tokenizer.decode(outputs[0]))

Input length of input_ids is 17, but ``max_length`` is set to 1. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 18, but ``max_length`` is set to 1. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 19, but ``max_length`` is set to 1. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 20, but ``max_length`` is set to 1. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Input length of input_ids is 21, but ``max_length`` is set to 1. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


Hi (Faster than 100k)

Downloads: 11,842

Hi (Faster than 100k)

Downloads: 11,842


Hi (Faster than 100k)

Downloads: 11,842

Total
Hi (Faster than 100k)

Downloads: 11,842

Total downloads
Hi (Faster than 100k)

Downloads: 11,842

Total downloads:


## PPO

In [ ]:
import gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Parallel environments
env = make_vec_env("CartPole-v1", n_envs=4)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)
model.save("ppo_cartpole")

del model # remove to demonstrate saving and loading

model = PPO.load("ppo_cartpole")

obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [2]:
from envs.AdaptEnv import AdaptEnv
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from models.GPT2RewardModel import GPT2RewardModel

device = 'cuda:0'

tokenizer = GPT2Tokenizer.from_pretrained('gpt2', pad_token='<|pad|>', device=device)
reward_model = GPT2RewardModel(device=device)

env = AdaptEnv('Tell me a joke.', 'Be funnier.', 100, reward_model , tokenizer, tokenizer.eos_token_id, device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from stable_baselines3 import PPO
from models.GPT2ActorCriticPolicy import GPT2ActorCriticPolicy

model = PPO(GPT2ActorCriticPolicy, env, device=device, verbose=1)

Using cuda:0 device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(11)

In [1]:
1+1

2